In [27]:
import pandas as pd
from sqlalchemy import create_engine

In [28]:
df_csv_src = pd.read_csv('sat-results.csv')
df_csv_src.head()
df_csv_src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              493 non-null    object 
 1   SCHOOL NAME                      493 non-null    object 
 2   Num of SAT Test Takers           493 non-null    object 
 3   SAT Critical Reading Avg. Score  493 non-null    object 
 4   SAT Math Avg. Score              493 non-null    object 
 5   SAT Writing Avg. Score           493 non-null    object 
 6   SAT Critical Readng Avg. Score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB


normalize column names (lowercase, replace: ' ', '.')

In [29]:
df_csv_src.columns = df_csv_src.columns.str.lower()
df_csv_src.columns = df_csv_src.columns.str.replace(" ", "_")
df_csv_src.columns = df_csv_src.columns.str.replace(r'[^a-z0-9_]', '', regex=True)
df_csv_src.columns

Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score', 'sat_critical_readng_avg_score',
       'internal_school_id', 'contact_extension', 'pct_students_tested',
       'academic_tier_rating'],
      dtype='object')

dupe check + dupe deletion

In [30]:
df_csv_src.duplicated().sum()
df_deduped = df_csv_src.drop_duplicates()
df_deduped.count()

dbn                               478
school_name                       478
num_of_sat_test_takers            478
sat_critical_reading_avg_score    478
sat_math_avg_score                478
sat_writing_avg_score             478
sat_critical_readng_avg_score     478
internal_school_id                478
contact_extension                 378
pct_students_tested               363
academic_tier_rating              392
dtype: int64

row_count: 493 > 478
try to automatically convert datatype


In [31]:
df_test = df_deduped.convert_dtypes()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 0 to 477
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   dbn                             478 non-null    string
 1   school_name                     478 non-null    string
 2   num_of_sat_test_takers          478 non-null    string
 3   sat_critical_reading_avg_score  478 non-null    string
 4   sat_math_avg_score              478 non-null    string
 5   sat_writing_avg_score           478 non-null    string
 6   sat_critical_readng_avg_score   478 non-null    string
 7   internal_school_id              478 non-null    Int64 
 8   contact_extension               378 non-null    string
 9   pct_students_tested             363 non-null    string
 10  academic_tier_rating            392 non-null    Int64 
dtypes: Int64(2), string(9)
memory usage: 45.7 KB


Result: object dtypes got converted to string.

Assessment:
- dbn: okay, since there's alway a char in the middle
- school_name: okay
- num_of_sat_test_takers: num expected
- sat_critical_reading_avg_score: num expected
- sat_math_avg_score: num expected
- sat_writing_avg_score: num expected
- sat_critical_readng_avg_score: same as 'sat_critical_reading_avg_score'? if yes, drop
- internal_school_id: okay
- contact_extension: num expected
- pct_students_tested: num expexted
- academic_tier_rating: okay

Check why sat related columns are interpreted as strings

In [32]:
df_test.groupby('sat_critical_reading_avg_score').count().sort_values(by=['sat_critical_reading_avg_score'], ascending=False)

,dbn,school_name,num_of_sat_test_takers,sat_math_avg_score,sat_writing_avg_score,sat_critical_readng_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
sat_critical_reading_avg_score,,,,,,,,,,
s,57,57,57,57,57,57,57,44,46,40
679,1,1,1,1,1,1,1,0,0,1
636,1,1,1,1,1,1,1,1,0,1
635,1,1,1,1,1,1,1,1,0,1
632,1,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
305,1,1,1,1,1,1,1,0,0,0
304,2,2,2,2,2,2,2,2,2,2
300,1,1,1,1,1,1,1,1,1,0


In [33]:
df_test.query('num_of_sat_test_takers == "s" and sat_critical_reading_avg_score == "s" and sat_math_avg_score == "s" and sat_writing_avg_score == "s"')
#.shape

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,sat_critical_readng_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
22,02M392,MANHATTAN BUSINESS ACADEMY,s,s,s,s,s,501072,<NA>,85%,<NA>
23,02M393,BUSINESS OF SPORTS SCHOOL,s,s,s,s,s,143204,x234,<NA>,2
25,02M399,THE HIGH SCHOOL FOR LANGUAGE AND DIPLOMACY,s,s,s,s,s,371710,x123,85%,<NA>
38,02M427,MANHATTAN ACADEMY FOR ARTS & LANGUAGE,s,s,s,s,s,521180,x234,<NA>,4
40,02M437,HUDSON HIGH SCHOOL OF LEARNING TECHNOLOGIES,s,s,s,s,s,254767,x345,<NA>,2
41,02M438,INTERNATIONAL HIGH SCHOOL AT UNION SQUARE,s,s,s,s,s,259655,x123,78%,1
47,02M473,WASHINGTON IRVING YABC,s,s,s,s,s,563977,x345,92%,3
77,03M402,THE URBAN ASSEMBLY SCHOOL FOR GREEN CAREERS,s,s,s,s,s,699396,x345,78%,3
78,03M403,THE GLOBAL LEARNING COLLABORATIVE,s,s,s,s,s,453954,x345,78%,1
81,03M417,FRANK MCCOURT HIGH SCHOOL,s,s,s,s,s,355018,x345,85%,4


57 rows mainly contain 's', possible stands for 'surpressed'. We can safely drop these rows

In [34]:
df_dropped_s = df_test[df_test['num_of_sat_test_takers'] != 's']

In [35]:
df_dropped_s.shape

(421, 11)

check if columns 'sat_critical_reading_avg_score' and 'sat_critical_readng_avg_score', if yes, then drop the column with typo

In [36]:
df_dropped_s.query('sat_critical_reading_avg_score != sat_critical_readng_avg_score')

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,sat_critical_readng_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating


In [37]:
df_cleaned_stage1 = df_dropped_s.drop(columns='sat_critical_readng_avg_score')
df_cleaned_stage1.columns

Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score', 'internal_school_id', 'contact_extension',
       'pct_students_tested', 'academic_tier_rating'],
      dtype='object')

Now, we have 421 columns remaining
Let Pandas interpret the datatypes again

In [38]:
df_cleaned_stage2 = df_cleaned_stage1.convert_dtypes()
df_cleaned_stage2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   dbn                             421 non-null    string
 1   school_name                     421 non-null    string
 2   num_of_sat_test_takers          421 non-null    string
 3   sat_critical_reading_avg_score  421 non-null    string
 4   sat_math_avg_score              421 non-null    string
 5   sat_writing_avg_score           421 non-null    string
 6   internal_school_id              421 non-null    Int64 
 7   contact_extension               334 non-null    string
 8   pct_students_tested             317 non-null    string
 9   academic_tier_rating            352 non-null    Int64 
dtypes: Int64(2), string(8)
memory usage: 37.0 KB


Since, convert_dtypes did not change anything, let's convert them explicitly

In [39]:
numeric_columns = [
    'num_of_sat_test_takers',
    'sat_critical_reading_avg_score', 
    'sat_math_avg_score',
    'sat_writing_avg_score'
]

# Create a copy and convert numeric columns
df_cleaned_stage2[numeric_columns] = df_cleaned_stage2[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Then let pandas optimize the remaining columns
df_cleaned_stage3 = df_cleaned_stage2.convert_dtypes()

# Check the result
df_cleaned_stage3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   dbn                             421 non-null    string
 1   school_name                     421 non-null    string
 2   num_of_sat_test_takers          421 non-null    Int64 
 3   sat_critical_reading_avg_score  421 non-null    Int64 
 4   sat_math_avg_score              421 non-null    Int64 
 5   sat_writing_avg_score           421 non-null    Int64 
 6   internal_school_id              421 non-null    Int64 
 7   contact_extension               334 non-null    string
 8   pct_students_tested             317 non-null    string
 9   academic_tier_rating            352 non-null    Int64 
dtypes: Int64(6), string(4)
memory usage: 38.6 KB


In [40]:
df_cleaned_stage3[df_cleaned_stage3['pct_students_tested'].isna()
].head(5)

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,268547,x234,<NA>,3
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,236446,x123,<NA>,3
5,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHOOL,112,332,557,316,414951,x345,<NA>,3
8,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624,604,628,881396,x234,<NA>,<NA>
15,02M303,"FACING HISTORY SCHOOL, THE",76,353,358,340,612918,<NA>,<NA>,<NA>


Since without the pct_students_tested the whole row does not give us relevant information, we should drop these rows

In [41]:
df_cleaned_stage4 = df_cleaned_stage3[~df_cleaned_stage3['pct_students_tested'].isna()]

In [42]:
df_cleaned_stage4.shape

(317, 10)

Same goes for 'academic_tier_rating'

In [43]:
df_cleaned_stage5 = df_cleaned_stage4[~df_cleaned_stage4['academic_tier_rating'].isna()]

In [44]:
df_cleaned_stage5

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,218160,x345,78%,2
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,427826,x123,92%,4
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,672714,x123,92%,2
6,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522,574,525,697107,<NA>,78%,2
7,01M650,CASCADES HIGH SCHOOL,18,417,418,411,297600,<NA>,92%,4
...,...,...,...,...,...,...,...,...,...,...
464,32K552,ACADEMY OF URBAN PLANNING,67,342,364,354,695572,x234,92%,3
466,32K556,BUSHWICK LEADERS HIGH SCHOOL FOR ACADEMIC EXCE...,23,347,358,350,658210,<NA>,92%,2
467,32K564,BUSHWICK COMMUNITY HIGH SCHOOL,24,359,317,358,898918,<NA>,92%,4
471,75Q811,P.S. Q811,32,429,444,433,467937,x123,78%,1


Now, we are at 269 rows, let's identify sat related values, which are out of range

In [45]:
df_cleaned_stage5.query('sat_critical_reading_avg_score < 200 or sat_critical_reading_avg_score > 800 or sat_math_avg_score < 200 or sat_math_avg_score > 800 or sat_writing_avg_score < 200 or sat_writing_avg_score > 800 ')

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
80,03M415,WADLEIGH SECONDARY SCHOOL FOR THE PERFORMING &...,32,371,850,370,365679,<NA>,78%,4
422,28Q470,JAMAICA HIGH SCHOOL,90,342,999,353,587904,x234,92%,3
434,29Q283,PREPARATORY ACADEMY FOR WRITERS: A COLLEGE BOA...,43,370,1100,363,462173,x345,85%,3


Result, only column 'sat_math_avg_score' is affected with 3 rows, let's remove these rows

In [46]:
df_cleaned_stage6 = df_cleaned_stage5.query('not (sat_critical_reading_avg_score < 200 or sat_critical_reading_avg_score > 800 or sat_math_avg_score < 200 or sat_math_avg_score > 800 or sat_writing_avg_score < 200 or sat_writing_avg_score > 800)')

In [47]:
df_cleaned_stage6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 0 to 476
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   dbn                             266 non-null    string
 1   school_name                     266 non-null    string
 2   num_of_sat_test_takers          266 non-null    Int64 
 3   sat_critical_reading_avg_score  266 non-null    Int64 
 4   sat_math_avg_score              266 non-null    Int64 
 5   sat_writing_avg_score           266 non-null    Int64 
 6   internal_school_id              266 non-null    Int64 
 7   contact_extension               214 non-null    string
 8   pct_students_tested             266 non-null    string
 9   academic_tier_rating            266 non-null    Int64 
dtypes: Int64(6), string(4)
memory usage: 24.4 KB



- col 'contact_extension'

Clean column 'pct_students_tested':
- remove '%'
- convert to float and convert to percent

In [48]:
df_cleaned_stage6.loc[:, 'pct_students_tested'] = df_cleaned_stage6['pct_students_tested'].str.replace("%","")

df_cleaned_stage6.loc[:,'pct_students_tested'] = pd.to_numeric(df_cleaned_stage6['pct_students_tested'].astype(str), errors='coerce') / 100


/var/folders/w8/9sd03p8d3msgg0z4xqqw1cbr0000gn/T/ipykernel_60653/2649865181.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.78 0.92 0.92 0.78 0.92 0.78 0.78 0.92 0.78 0.78 0.78 0.92 0.92 0.85
 0.92 0.92 0.85 0.78 0.92 0.92 0.92 0.85 0.92 0.85 0.85 0.78 0.92 0.92
 0.85 0.78 0.85 0.78 0.78 0.92 0.85 0.85 0.85 0.78 0.85 0.78 0.92 0.92
 0.85 0.78 0.92 0.92 0.92 0.92 0.85 0.92 0.78 0.92 0.92 0.85 0.78 0.92
 0.92 0.78 0.92 0.78 0.78 0.85 0.85 0.92 0.92 0.78 0.78 0.92 0.92 0.78
 0.78 0.78 0.78 0.92 0.85 0.92 0.78 0.85 0.85 0.92 0.78 0.78 0.78 0.78
 0.78 0.92 0.85 0.92 0.78 0.78 0.78 0.85 0.85 0.78 0.85 0.85 0.92 0.85
 0.78 0.85 0.85 0.92 0.85 0.85 0.92 0.78 0.92 0.92 0.85 0.85 0.78 0.78
 0.78 0.78 0.78 0.85 0.92 0.78 0.78 0.78 0.85 0.85 0.92 0.85 0.85 0.85
 0.85 0.85 0.78 0.85 0.92 0.78 0.78 0.78 0.92 0.92 0.92 0.92 0.78 0.78
 0.78 0.78 0.78 0.78 0.85 0.92 0.85 0.78 0.92 0.92 0.85 0.85 0.92 0.85
 0.92 0.78 0.78 0.9

Check if warning had any effects on the column:

In [49]:
df_cleaned_stage6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 0 to 476
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             266 non-null    string 
 1   school_name                     266 non-null    string 
 2   num_of_sat_test_takers          266 non-null    Int64  
 3   sat_critical_reading_avg_score  266 non-null    Int64  
 4   sat_math_avg_score              266 non-null    Int64  
 5   sat_writing_avg_score           266 non-null    Int64  
 6   internal_school_id              266 non-null    Int64  
 7   contact_extension               214 non-null    string 
 8   pct_students_tested             266 non-null    float64
 9   academic_tier_rating            266 non-null    Int64  
dtypes: Int64(6), float64(1), string(3)
memory usage: 24.4 KB


Pct_students_tested looks good, finally, let's check column 'contact_extension':

In [50]:
df_cleaned_stage6['contact_extension'].value_counts()

contact_extension
x345    78
x234    70
x123    66
Name: count, dtype: Int64

'contact_extension' contains 3 distinct values and does not add value to the analysis, we'll delete this column

check analytical value of column 'academic_tier_rating'

check analytical value of column 'academic_tier_rating'

In [26]:
df_cleaned_stage6['academic_tier_rating'].value_counts()

academic_tier_rating
4    77
2    68
1    61
3    60
Name: count, dtype: Int64

'academic_tier_rating' contains 4 distinct values, we'll drop this column too

In [56]:
df_final = df_cleaned_stage6.drop(columns=['contact_extension', 'academic_tier_rating'])
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 0 to 476
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             266 non-null    string 
 1   school_name                     266 non-null    string 
 2   num_of_sat_test_takers          266 non-null    Int64  
 3   sat_critical_reading_avg_score  266 non-null    Int64  
 4   sat_math_avg_score              266 non-null    Int64  
 5   sat_writing_avg_score           266 non-null    Int64  
 6   internal_school_id              266 non-null    Int64  
 7   pct_students_tested             266 non-null    float64
dtypes: Int64(5), float64(1), string(2)
memory usage: 20.0 KB


### Result: Now we have a dataframe with zero missing values containing 8 columns and 266 rows

Save current dataframe to csv

In [60]:
df_final.to_csv('cleaned_sat_results.csv', index=False)

Create table on postgres db

In [58]:
host = "ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech"
port = "5432"
database = "neondb"
user = "neondb_owner"
password = "a9Am7Yy5r9_T7h4OF2GN"

# Create a connection engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

In [59]:
df_final.to_sql(
    name='michael_sat_results',       
    con=engine,     
    schema='nyc_schools',
    if_exists='replace',    
    index=False            
)

266